# Reading the data

In [2]:
import pandas as pd
import numpy as np
import shutil
import glob
import os
import sys
import pickle
import datetime as dt
import re
from collections import Counter
from math import radians, cos, sin, asin, sqrt
import calendar
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


I loop through the file path to combine seperate csvs into one dataset

In [27]:
# os.listdir()
# path = r"bike_data/"

# all_data = glob.glob(path + '/*.csv')
# with open('mb.csv', 'wb') as outfile:
#     for i, fname in enumerate(all_data):
#         with open(fname, 'rb') as infile:
#             if i != 0:
#                 infile.readline() 
#             shutil.copyfileobj(infile, outfile)
#             print(fname + ' imported.')

bike_data\OD_2014-04.csv has been imported.
bike_data\OD_2014-05.csv has been imported.
bike_data\OD_2014-06.csv has been imported.
bike_data\OD_2014-07.csv has been imported.
bike_data\OD_2014-08.csv has been imported.
bike_data\OD_2014-09.csv has been imported.
bike_data\OD_2014-10.csv has been imported.
bike_data\OD_2014-11.csv has been imported.
bike_data\OD_2015-04.csv has been imported.
bike_data\OD_2015-05.csv has been imported.
bike_data\OD_2015-06.csv has been imported.
bike_data\OD_2015-07.csv has been imported.
bike_data\OD_2015-08.csv has been imported.
bike_data\OD_2015-09.csv has been imported.
bike_data\OD_2015-10.csv has been imported.
bike_data\OD_2015-11.csv has been imported.
bike_data\OD_2016-04.csv has been imported.
bike_data\OD_2016-05.csv has been imported.
bike_data\OD_2016-06.csv has been imported.
bike_data\OD_2016-07.csv has been imported.
bike_data\OD_2016-08.csv has been imported.
bike_data\OD_2016-09.csv has been imported.
bike_data\OD_2016-10.csv has bee

In [4]:
#  output from above
mb = pd.read_csv('mb.csv')
mb.head()
# mb = mb.drop(['Unnamed: 0'], axis=1)

,Unnamed: 0,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,138,2014-04-15 07:53,6184,2014-04-15 08:02,6015,517,1
1,3257,2014-04-16 16:55,6184,2014-04-16 17:06,6015,686,1
2,3773,2014-04-16 18:11,6184,2014-04-16 18:18,6015,439,1
3,4028,2014-04-16 18:53,6184,2014-04-16 19:03,6015,580,1
4,5672,2014-04-17 08:58,6184,2014-04-17 09:05,6015,412,1


gets the required station to station route

In [5]:
df = mb
start = ['6184']
end = ['6015']

mb = df[(df['start_station_code'].isin(start)) & (df['end_station_code'].isin(end))].sort_values(by='start_date')

# Creating features in the dataset

calculating average speed by finding route

In [6]:
ll = ['6184','6015']
locations = pd.read_csv('Stations_2017.csv')
locations.iloc
locations = locations[(locations['code'].isin(ll))].sort_values(by='name', ascending = False)
locations

location = locations['latitude'].mean(), locations['longitude'].mean()

viewing route distance

In [2]:
import googlemaps

gmaps = googlemaps.Client(key='ENTER GOOGLE API KEY')
now = dt.datetime.now()
st1 = "45.524673, -73.582550"
st2 = "45.515299, -73.561273"
directions_out = gmaps.directions(st1,st2,
                                     mode="bicycling",
                                     departure_time=now)
print('distance: '+ directions_out[0]['legs'][0]['distance']['text'])
print('expected time: '+ directions_out[0]['legs'][0]['duration']['text'])

distance: 2.2 km
expected time: 8 mins


adding features

In [7]:
distance_km = 2.2

# adding features to the data-set
from datetime import datetime

mb["date"] = mb.start_date.apply(lambda x : x.split()[0])
mb["start_hour"] = mb.start_date.apply(lambda x : x.split()[1].split(':')[0])
mb["end_hour"] = mb.end_date.apply(lambda x : x.split()[1].split(':')[0])
mb["weekday"] = mb.date.apply(lambda x : calendar.day_name[datetime.strptime(x,"%Y-%m-%d").weekday()])
mb["month"] = mb.date.apply(lambda x : calendar.month_name[datetime.strptime(x,"%Y-%m-%d").month])
mb['date'] = pd.to_datetime(mb['date'])
mb['year'] = mb.date.map(lambda x: x.year)
mb['start_date'] = pd.to_datetime(mb['start_date'])

mb['season']= mb.start_date.apply(lambda z: (z.month%12 + 3)//3)
mb['season'] = mb.season.astype(str)
mb['season'] = mb.season.apply(lambda x: re.sub('.',lambda m: {'1':'Autumn','2':'Spring','3':'Summer','4':'Winter'}
                                                .get(m.group(), m.group()),x))

mb['start_hour'] = pd.to_datetime(mb['start_hour'], format = '%H', errors='coerce')
mb = mb.assign(time_of_day=pd.cut(mb.start_hour.dt.hour,[-1,6,12,18,24],
                                  labels=['Night','Morning','Afternoon','Evening']))
mb['start_hour'] = mb.start_hour.dt.hour
mb['end_hour'] = pd.to_datetime(mb['end_hour'], format = '%H', errors='coerce')
mb['end_hour'] = mb.end_hour.dt.hour

mb['avg_speed'] = (distance_km / (mb.loc[:,'duration_sec']/3600)).round(2)

mb
snpshot = mb


reasonableness checks

In [10]:
len(mb)

Counter(mb[mb['start_station_code']== 6184
          ]['start_station_code'])

# mb = mb.drop(['Unnamed: 0'], axis=1)

mb.shape
mb.head()


Counter({6184: 2173})

# Visualisation

seeing the journey

In [8]:
import folium

locationlist = locations[['latitude','longitude']].values.tolist()
labels = locations['name'].values.tolist()

m = folium.Map(location=location, zoom_start=14)
for point in range(len(locationlist)):
    popup = folium.Popup(labels[point], parse_html=True)
    folium.Marker(locationlist[point], popup=popup).add_to(m)

line = folium.PolyLine(
    [[45.524673, -73.582550],
     [45.515299, -73.561273]]).add_to(m)

m

journeys by season

In [9]:
# member stats

dc = Counter(mb[mb['is_member'] == 1]['season'])
df = pd.DataFrame.from_dict(dc, orient='index').reset_index()
df.columns = ['seasons','journeys']
total = df.journeys.values.sum()
df['%'] = (df.journeys.div(total) * 100).round()
df

,seasons,journeys,%
0,Spring,322,18.0
1,Summer,905,50.0
2,Winter,581,32.0


In [10]:
df.iplot(kind='barh',x='seasons',y='journeys', theme='white',
         colors='red',xTitle='Count',title='Journeys by Season')



journeys by day of week

In [11]:
journeys_per_day = mb.weekday.value_counts().reset_index()
journeys_per_day.columns = ['day','day_of_week']
#  formatting
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
mapping = {day: i for i, day in enumerate(days)}
key = journeys_per_day['day'].map(mapping)

journeys_per_day = journeys_per_day.iloc[key.argsort()].set_index('day')

In [12]:
journeys_per_day.iplot(kind='bar',theme='white',colors='blue',
                       title='Journeys by day of week',xTitle='Weekday',yTitle='Journeys')


### memeber data

In [13]:
member = mb[mb['is_member']==1]
non_member = mb[mb['is_member'] == 0]

non_member.head()

,Unnamed: 0,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,date,start_hour,end_hour,weekday,month,year,season,time_of_day,avg_speed
156,516750,2014-05-29 16:11:00,6184,2014-05-29 16:21,6015,615,0,2014-05-29,16,16,Thursday,May,2014,Spring,Afternoon,12.88
161,525337,2014-05-29 21:44:00,6184,2014-05-29 21:52,6015,479,0,2014-05-29,21,21,Thursday,May,2014,Spring,Evening,16.53
167,534243,2014-05-30 14:17:00,6184,2014-05-30 14:28,6015,648,0,2014-05-30,14,14,Friday,May,2014,Spring,Afternoon,12.22
168,534253,2014-05-30 14:19:00,6184,2014-05-30 14:28,6015,585,0,2014-05-30,14,14,Friday,May,2014,Spring,Afternoon,13.54
207,655764,2014-06-06 09:57:00,6184,2014-06-06 10:07,6015,623,0,2014-06-06,9,10,Friday,June,2014,Summer,Morning,12.71


In [12]:
mb.avg_speed.max().round()

27.0

In [13]:
mb.avg_speed.min()

1.46

average speeds of members vs non-members by time of day

In [14]:
mb.pivot_table(index='is_member',columns='time_of_day',values='avg_speed')

time_of_day,Night,Morning,Afternoon,Evening
is_member,,,,
0,13.303611,11.282273,10.476458,10.501443
1,15.259310,14.209474,13.754844,14.137976


average speed by season for members vs non-members

In [15]:
mb.pivot_table(index='is_member',columns='season',values='avg_speed')

season,Spring,Summer,Winter
is_member,,,
0,10.388980,10.879878,11.611127
1,13.973075,13.868265,14.260138


distribution of average speed for members vs non-members by hours of the day

In [16]:
mb.is_member.replace([0,1],['non-member','member'],inplace=True)
pltmb = mb.pivot_table(index='is_member',columns='start_hour',values='avg_speed')
pltmb.iplot(kind='bar',theme='white', xTitle='hour of the day', yTitle='average speed (km/h)',
            title='distribution of average speed for members vs non-members by hours of the day')


In [17]:
mpvt = member.pivot_table(values='is_member', columns='month', 
                         aggfunc='count')

non_member.rename(columns = {'is_member':'not_member'}, inplace = True) 

nmpvt = non_member.pivot_table(values='not_member', columns='month', 
                         aggfunc='count')

mpvt = mpvt.append(nmpvt, ignore_index=False)
MPVT = mpvt.T.reset_index()

/Users/Ab/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [18]:
MPVT.iplot(kind='line',x='month',y='not_member', color='black', theme='white',
                    mode='lines',width=2.4
                    ,title='bike demand by month for non-members',yTitle='Journeys')

MPVT.iplot(kind='line',x='month',y='is_member', color='gold', theme = 'white',width=2.4,
                    mode='lines', title='bike demand by month for non-members', yTitle='Journeys')

In [112]:
mb.columns

Index(['start_date', 'start_station_code', 'end_date', 'end_station_code',
       'duration_sec', 'is_member', 'date', 'start_hour', 'end_hour',
       'weekday', 'month', 'year', 'season', 'time_of_day', 'avg_speed'],
      dtype='object')

#### durations in sec by date

In [19]:
mb.iplot(kind='scatter',y='duration_sec',x='date',theme='white',color='red')

five day moving average on journey durations

In [20]:
day_average_mb = pd.DataFrame(mb.groupby(['year', 'month', 'weekday'],as_index=False)['duration_sec'].mean())
day_average_member =pd.DataFrame(member.groupby(['year', 'month', 'weekday'],as_index=False)['duration_sec'].mean())
day_average_non_member =pd.DataFrame(non_member.groupby(['year', 'month', 'weekday'],as_index=False)['duration_sec'].mean())


In [24]:
mb_gb = pd.DataFrame(day_average_mb.groupby(['month'],as_index=False)['duration_sec'].mean())
mb_gb.sort_values('month', ascending=True, inplace=True)
mb_gb['five_day_avg']=pd.Series(mb_gb['duration_sec'].rolling(window = 5, min_periods = 1).mean())

mb_gb.iplot(kind='line',x='month',y='five_day_avg',theme='white',color='blue',mode='markers+lines',width=2.4,title='Five day moving average of duration per second by months')

In [25]:
monthly_mb = day_average_mb.groupby(['month'], as_index=False)['duration_sec'].mean()
monthly_member = day_average_member.groupby(['month'], as_index=False)['duration_sec'].mean()
monthly_non_member = day_average_non_member.groupby(['month'], as_index=False)['duration_sec'].mean()

In [29]:
monthly_mb.iplot(kind='scatter', x='month', y='duration_sec', 
                         title = 'total',theme='white',color='teal',width=2.4)

monthly_member.iplot(kind='scatter', x='month', y='duration_sec', 
                         title = 'member',theme='white',color='purple',width=2.4)



monthly_non_member.iplot(kind='scatter', x='month', y='duration_sec', 
                         title = 'non-member',theme='white',color='navy',width=2.4)



moving forward, an exploration of running a predictive analysis based on training data to predict average speeds and journey durations based on time of day, hour, season, month etc.

this analysis would be useful for Bixi as they would gain a better grasp on understanding bike usage. Information which can directly feed into bike supply at specific stations and ease the operational worries of predicting the right time to service a bike (analysis paired with kilometer limit on bikes from manufacturer)